In [1]:
import sys
sys.path.append('../../')

In [2]:
from models.refinenet_resnet import refinenet_resnet101
pretrained_weight_dir = '../../pretrained/Cityscapes'
model = refinenet_resnet101(pretrained_weight_dir).eval()
model_name = '{}({})'.format(str(model).split('(')[0], str(model.resnet).split('(')[0])
model_name

'RefineNet_ResNet(Resnet101)'

In [3]:
import torch
import datasets
from torchvision import transforms
import numpy as np

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda img: img*255)])
target_transform = transforms.Lambda(lambda img: torch.from_numpy(np.array(img)))

foggy_cityscapes = datasets.EnvCityscapes(
    '/workbench/data/Cityscapes', split='train',
    env=["", 'foggyDBF', 'transmittanceDBF'],
    env_type=[[''], ['beta_0.01'], ['beta_0.01']],
    target_type=['semantic', 'color'],
    transform=transforms.Lambda(lambda imgs: tuple(map(transform, imgs))),
    target_transform=transforms.Lambda(lambda imgs: tuple(map(target_transform, imgs))),
)
foggy_cityscapes_val = datasets.EnvCityscapes(
    '/workbench/data/Cityscapes', split='val',
    env=["", 'foggyDBF', 'transmittanceDBF'],
    env_type=[[''], ['beta_0.01'], ['beta_0.01']],
    target_type=['semantic', 'color'],
    transform=transforms.Lambda(lambda imgs: tuple(map(transform, imgs))),
    target_transform=transforms.Lambda(lambda imgs: tuple(map(target_transform, imgs))),
)
zurich_testv2 = datasets.Zurich(
    '/workbench/data/Foggy_Zurich', split='testv2',
    target_type=['gt_labelIds', 'RGB'],
    transform=transform,
    target_transform=transforms.Lambda(lambda imgs: tuple(map(target_transform, imgs))),
)

with open('/workbench/data/Cityscapes/foggy_trainval_refined_filenames.txt') as f:
    trainval_refined_filenames = sorted([line.strip() for line in f.readlines()])

images, targets = map(np.array, [foggy_cityscapes.images, foggy_cityscapes.targets])
refine_mask = np.zeros(images.shape[0]).astype(np.bool)
for i, image in enumerate(images):
    for filename in trainval_refined_filenames:
        if filename in image[0]:
            refine_mask[i] = True
            break
foggy_cityscapes.images, foggy_cityscapes.targets = images[refine_mask], targets[refine_mask]

images, targets = map(np.array, [foggy_cityscapes_val.images, foggy_cityscapes_val.targets])
refine_mask = np.zeros(images.shape[0]).astype(np.bool)
for i, image in enumerate(images):
    for filename in trainval_refined_filenames:
        if filename in image[0]:
            refine_mask[i] = True
foggy_cityscapes_val.images, foggy_cityscapes_val.targets = images[refine_mask], targets[refine_mask]

dataset_name = str(foggy_cityscapes).split()[1]
dataset_name

'EnvCityscapes'

In [4]:
classes = datasets.Cityscapes.classes
id2label = {cls.id:cls for cls in classes}
train_id2label = {cls.train_id:cls for cls in classes}
pred_argmax2label = {cls.train_id:cls for cls in classes}
pred_argmax2label[19] = pred_argmax2label[255]
pred_argmax2label.pop(255);

In [5]:
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from utils.metrics import runningScore

foggy_cityscapes_val_running_metrics = runningScore(20)
foggy_zurich_testv2_running_metrics = runningScore(20)

In [6]:
# train_loader = DataLoader(foggy_cityscapes)
valid_loader = DataLoader(foggy_cityscapes_val)
test_loader = DataLoader(zurich_testv2)

model = nn.DataParallel(model.cuda(), device_ids=[0, 1]).eval()
criteria = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=5e-5)

with torch.no_grad():
    epoch_loss = 0
    for i, ((clear, foggy, trans), (labelId, color)) in enumerate(tqdm(valid_loader)):
        logit = model(F.interpolate(foggy, None, 0.7, mode='bilinear')-128).cpu()
        gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
        gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
        gt = torch.from_numpy(gt)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
        loss = criteria(scaled_logit, gt)

        epoch_loss += loss

        foggy_cityscapes_val_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#         break
    print({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})
#     wandb.log({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})

    scores = foggy_cityscapes_val_running_metrics.get_scores()
    for k, v in scores[0].items():
        print({'Valid_{}'.format(k): v})
#         wandb.log({'Valid_{}'.format(k): v})
    print({'Valid_IoU': list(scores[1].values())})
#     wandb.log({'Valid_IoU': list(scores[1].values())})

    epoch_loss = 0
    for i, (foggy, (labelId, color)) in enumerate(tqdm(test_loader)):
        logit = model(F.interpolate(foggy, None, 0.7, mode='bilinear')-128).cpu()
        gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
        gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
        gt = torch.from_numpy(gt)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
        loss = criteria(scaled_logit, gt)

        epoch_loss += loss

        foggy_zurich_testv2_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#         break
    print({'Test_Epoch_Loss': epoch_loss / len(test_loader)})
#     wandb.log({'Test_Epoch_Loss': epoch_loss / len(test_loader)})

    scores = foggy_zurich_testv2_running_metrics.get_scores()
    for k, v in scores[0].items():
        print({'Test_{}'.format(k): v})
#         wandb.log({'Test_{}'.format(k): v})
    print({'Test_IoU': list(scores[1].values())})
#     wandb.log({'Test_IoU': list(scores[1].values())})        

  0%|          | 0/52 [00:00<?, ?it/s]/opt/conda/envs/py37torch1.5/lib/python3.8/site-packages/torch/nn/functional.py:2970: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "
/opt/conda/envs/py37torch1.5/lib/python3.8/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/opt/conda/conda-bld/pytorch_158742820743

{'Valid_Epoch_Loss': tensor(1.5168)}
{'Valid_Overall Acc: \t': 0.85252989255465}
{'Valid_Mean Acc : \t': 0.6836295078542797}
{'Valid_FreqW Acc : \t': 0.7396877622958592}
{'Valid_Mean IoU : \t': 0.5520931930837549}
{'Valid_IoU': [0.860120478505304, 0.7628360661628238, 0.79230846880749, 0.3290042536298628, 0.2276113937523802, 0.2780178900106259, 0.4798609939250759, 0.5321800445539553, 0.7904910169261211, 0.6079896895166883, 0.7224032122295665, 0.6339571095955508, 0.5388180394413015, 0.8679975880435621, 0.6683743714797471, 0.7415572393458361, nan, 0.0, 0.6562428126654534, 0.0]}


100%|██████████| 40/40 [01:04<00:00,  1.62s/it]

{'Test_Epoch_Loss': tensor(3.7494)}
{'Test_Overall Acc: \t': 0.6366000916280864}
{'Test_Mean Acc : \t': 0.4831073520131504}
{'Test_FreqW Acc : \t': 0.48205167698985324}
{'Test_Mean IoU : \t': 0.30629537693110176}
{'Test_IoU': [0.3849680954601358, 0.42970536482290966, 0.4511040231211075, 0.23142379163035165, 0.2619096111026031, 0.3837427945336064, 0.4932149438766963, 0.500637611788856, 0.638463106108425, 0.2813025968351665, 0.8157585521848876, 0.03976025161711471, 0.18444100978876868, 0.5295348502241292, 0.0, 0.0, 0.0, 0.47093936537857367, 0.02900157014870232, 0.0]}


In [7]:
foggy_cityscapes_val_running_metrics = runningScore(20)
foggy_zurich_testv2_running_metrics = runningScore(20)

with torch.no_grad():
    epoch_loss = 0
    for i, ((clear, foggy, trans), (labelId, color)) in enumerate(tqdm(valid_loader)):
        logit = model(F.interpolate(foggy, None, 0.8*1.2, mode='bilinear')-128).cpu()
        gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
        gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
        gt = torch.from_numpy(gt)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
        loss = criteria(scaled_logit, gt)

        epoch_loss += loss

        foggy_cityscapes_val_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#         break
    print({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})
#     wandb.log({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})

    scores = foggy_cityscapes_val_running_metrics.get_scores()
    for k, v in scores[0].items():
        print({'Valid_{}'.format(k): v})
#         wandb.log({'Valid_{}'.format(k): v})
    print({'Valid_IoU': list(scores[1].values())})
#     wandb.log({'Valid_IoU': list(scores[1].values())})

    epoch_loss = 0
    for i, (foggy, (labelId, color)) in enumerate(tqdm(test_loader)):
        logit = model(F.interpolate(foggy, None, 0.8*1.2, mode='bilinear')-128).cpu()
        gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
        gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
        gt = torch.from_numpy(gt)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
        loss = criteria(scaled_logit, gt)

        epoch_loss += loss

        foggy_zurich_testv2_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#         break
    print({'Test_Epoch_Loss': epoch_loss / len(test_loader)})
#     wandb.log({'Test_Epoch_Loss': epoch_loss / len(test_loader)})

    scores = foggy_zurich_testv2_running_metrics.get_scores()
    for k, v in scores[0].items():
        print({'Test_{}'.format(k): v})
#         wandb.log({'Test_{}'.format(k): v})
    print({'Test_IoU': list(scores[1].values())})
#     wandb.log({'Test_IoU': list(scores[1].values())})        

  0%|          | 0/40 [00:00<?, ?it/s]

{'Valid_Epoch_Loss': tensor(1.5257)}
{'Valid_Overall Acc: \t': 0.8570143167789166}
{'Valid_Mean Acc : \t': 0.7267194116941995}
{'Valid_FreqW Acc : \t': 0.7450807493407154}
{'Valid_Mean IoU : \t': 0.567044502769608}
{'Valid_IoU': [0.8654596428669264, 0.7827728115753442, 0.7993688647376012, 0.37998627492529946, 0.26367975008334854, 0.43340587702367356, 0.5456701875312463, 0.5856431325097785, 0.7798650765781646, 0.5978799316028625, 0.6971544065671366, 0.7024777556410973, 0.6156265685126318, 0.8914462442503529, 0.711092319892744, 0.8070739549839229, 0.0, 0.17330175670004197, 0.7089854994099891, 0.0]}


100%|██████████| 40/40 [01:06<00:00,  1.67s/it]

{'Test_Epoch_Loss': tensor(4.0448)}
{'Test_Overall Acc: \t': 0.5756634717399691}
{'Test_Mean Acc : \t': 0.4637016449692234}
{'Test_FreqW Acc : \t': 0.41575008729299195}
{'Test_Mean IoU : \t': 0.2694536341874619}
{'Test_IoU': [0.3821877055092792, 0.4800701286041575, 0.3469410584212502, 0.20257132246801685, 0.22012535905468855, 0.33325139139929605, 0.4988683979100926, 0.5107303782877501, 0.5821353628293211, 0.2890542250205955, 0.6774765065393576, 0.028254827905650967, 0.2189922480620155, 0.32840232382600504, 0.0, 0.024031614924522927, 0.0, 0.20414821719878817, 0.06183161578845032, 0.0]}


In [8]:
foggy_cityscapes_val_running_metrics = runningScore(20)
foggy_zurich_testv2_running_metrics = runningScore(20)

with torch.no_grad():
    epoch_loss = 0
    for i, ((clear, foggy, trans), (labelId, color)) in enumerate(tqdm(valid_loader)):
        logit = model(F.interpolate(F.interpolate(foggy, None, 0.8, mode='bilinear'), None, 1.2, mode='bilinear').round()-128).cpu()
        gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
        gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
        gt = torch.from_numpy(gt)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
        loss = criteria(scaled_logit, gt)

        epoch_loss += loss

        foggy_cityscapes_val_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#         break
    print({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})
#     wandb.log({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})

    scores = foggy_cityscapes_val_running_metrics.get_scores()
    for k, v in scores[0].items():
        print({'Valid_{}'.format(k): v})
#         wandb.log({'Valid_{}'.format(k): v})
    print({'Valid_IoU': list(scores[1].values())})
#     wandb.log({'Valid_IoU': list(scores[1].values())})

    epoch_loss = 0
    for i, (foggy, (labelId, color)) in enumerate(tqdm(test_loader)):
        logit = model(F.interpolate(F.interpolate(foggy, None, 0.8, mode='bilinear'), None, 1.2, mode='bilinear').round()-128).cpu()
        gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
        gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
        gt = torch.from_numpy(gt)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='bilinear')
        loss = criteria(scaled_logit, gt)

        epoch_loss += loss

        foggy_zurich_testv2_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#         break
    print({'Test_Epoch_Loss': epoch_loss / len(test_loader)})
#     wandb.log({'Test_Epoch_Loss': epoch_loss / len(test_loader)})

    scores = foggy_zurich_testv2_running_metrics.get_scores()
    for k, v in scores[0].items():
        print({'Test_{}'.format(k): v})
#         wandb.log({'Test_{}'.format(k): v})
    print({'Test_IoU': list(scores[1].values())})
#     wandb.log({'Test_IoU': list(scores[1].values())})        

  0%|          | 0/40 [00:00<?, ?it/s]

{'Valid_Epoch_Loss': tensor(1.5058)}
{'Valid_Overall Acc: \t': 0.8519160289030808}
{'Valid_Mean Acc : \t': 0.7162401002146094}
{'Valid_FreqW Acc : \t': 0.7377891147121705}
{'Valid_Mean IoU : \t': 0.5847625828513909}
{'Valid_IoU': [0.8648271015248844, 0.7752323584159523, 0.7875850536624867, 0.36704517446227114, 0.21584515053428344, 0.36418532492985145, 0.5281143391803029, 0.5811362674003901, 0.7658419067883939, 0.5960035550266362, 0.6829193293799689, 0.6941320268858155, 0.6082552890492939, 0.8852515658125745, 0.7167966963317259, 0.8029642326591246, nan, 0.17797892776063798, 0.6963747743718353, 0.0]}


100%|██████████| 40/40 [01:10<00:00,  1.77s/it]

{'Test_Epoch_Loss': tensor(3.9137)}
{'Test_Overall Acc: \t': 0.5768236521026234}
{'Test_Mean Acc : \t': 0.4876289167799242}
{'Test_FreqW Acc : \t': 0.4181794700145351}
{'Test_Mean IoU : \t': 0.277838119931949}
{'Test_IoU': [0.3985490566934314, 0.48676612067306796, 0.34182929309936577, 0.22104101403551799, 0.22235711617482257, 0.34200403663611423, 0.500747729533712, 0.5183094131757277, 0.6053870715656398, 0.2934413877883565, 0.6720991373978886, 0.012552739851673262, 0.2020228215767635, 0.22684170106592222, 0.0, 0.10535418063741926, 0.0, 0.35466091540857897, 0.052798663324979116, 0.0]}


In [9]:
foggy_cityscapes_val_running_metrics = runningScore(20)
foggy_zurich_testv2_running_metrics = runningScore(20)

with torch.no_grad():
    epoch_loss = 0
    for i, ((clear, foggy, trans), (labelId, color)) in enumerate(tqdm(valid_loader)):
        logit = model(F.interpolate(foggy, None, 0.8*1.2, mode='nearest')-128).cpu()
        gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
        gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
        gt = torch.from_numpy(gt)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='nearest')
        loss = criteria(scaled_logit, gt)

        epoch_loss += loss

        foggy_cityscapes_val_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#         break
    print({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})
#     wandb.log({'Valid_Epoch_Loss': epoch_loss / len(valid_loader)})

    scores = foggy_cityscapes_val_running_metrics.get_scores()
    for k, v in scores[0].items():
        print({'Valid_{}'.format(k): v})
#         wandb.log({'Valid_{}'.format(k): v})
    print({'Valid_IoU': list(scores[1].values())})
#     wandb.log({'Valid_IoU': list(scores[1].values())})

    epoch_loss = 0
    for i, (foggy, (labelId, color)) in enumerate(tqdm(test_loader)):
        logit = model(F.interpolate(foggy, None, 0.8*1.2, mode='nearest')-128).cpu()
        gt = np.vectorize(lambda x: id2label[x].train_id)(labelId.numpy())
        gt = np.vectorize(lambda x: 19 if x == 255 else x)(gt)
        gt = torch.from_numpy(gt)
        scaled_logit = F.interpolate(logit, gt.shape[-2:], mode='nearest')
        loss = criteria(scaled_logit, gt)

        epoch_loss += loss

        foggy_zurich_testv2_running_metrics.update(gt.numpy(), scaled_logit.argmax(1).numpy())
#         break
    print({'Test_Epoch_Loss': epoch_loss / len(test_loader)})
#     wandb.log({'Test_Epoch_Loss': epoch_loss / len(test_loader)})

    scores = foggy_zurich_testv2_running_metrics.get_scores()
    for k, v in scores[0].items():
        print({'Test_{}'.format(k): v})
#         wandb.log({'Test_{}'.format(k): v})
    print({'Test_IoU': list(scores[1].values())})
#     wandb.log({'Test_IoU': list(scores[1].values())})        

  0%|          | 0/40 [00:00<?, ?it/s]

{'Valid_Epoch_Loss': tensor(1.5223)}
{'Valid_Overall Acc: \t': 0.8592595412180974}
{'Valid_Mean Acc : \t': 0.7257878844114699}
{'Valid_FreqW Acc : \t': 0.7485723431979625}
{'Valid_Mean IoU : \t': 0.5683453127049364}
{'Valid_IoU': [0.8656095034487464, 0.788225704584243, 0.8024034381699211, 0.37293171865666813, 0.28453587665725927, 0.4458622643142031, 0.5508895921989607, 0.5881193972781216, 0.7884486032304192, 0.5969351893038289, 0.7114372425156823, 0.7049977109029015, 0.625016939582157, 0.8942143470480808, 0.7199126968218971, 0.7986204499089753, 0.0, 0.1197485341231941, 0.7089970453534675, 0.0]}


100%|██████████| 40/40 [01:04<00:00,  1.60s/it]

{'Test_Epoch_Loss': tensor(4.1774)}
{'Test_Overall Acc: \t': 0.5718171657986111}
{'Test_Mean Acc : \t': 0.4487158175913341}
{'Test_FreqW Acc : \t': 0.4120746616689452}
{'Test_Mean IoU : \t': 0.26857005812757023}
{'Test_IoU': [0.3692738731966791, 0.4614336567688733, 0.34653323721949425, 0.19388557506414025, 0.2199939304199349, 0.32009378244852876, 0.49251239272094127, 0.4983308516259186, 0.5585877399788935, 0.2782991963647871, 0.6786939420064496, 0.03341318875299458, 0.19932001699957502, 0.4707306008661903, 0.0, 0.0, 0.0, 0.1926530612244898, 0.057646116893514815, 0.0]}
